# Feature Generation 
## 1.for sampled_play
### 1.1 Frequency of time_period, average play_time of the same time_period 

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
colname = ['uid', 'device', 'song_id', 'play_time', 'song_length', 'date', 'Churn']
datatype = {'uid': 'float64', 'device': 'object', 'song_id': 'float64', 'song_length': 'float64', 'date':'object', 'Churn': 'float64'}
sample_play = pd.read_csv("D:/MusicFile/sample_play_316.csv", header = 0, dtype = datatype, names = colname)

In [4]:
sample_play.head()

,uid,device,song_id,play_time,song_length,date,Churn
0,49423096.0,ar,203139.0,204.0,270.0,2017-04-02,0.0
1,49423096.0,ar,133946.0,165.0,286.0,2017-04-02,0.0
2,49423096.0,ar,380389.0,40.0,233.0,2017-04-02,0.0
3,49423096.0,ar,23534240.0,115.0,378.0,2017-04-02,0.0
4,49423096.0,ar,6672344.0,69.0,249.0,2017-04-02,0.0


In [7]:
sample_play['date'] = pd.to_datetime(sample_play['date'])
sample_play.dtypes

uid                   float64
device                 object
song_id               float64
play_time             float64
song_length           float64
date           datetime64[ns]
Churn                 float64
dtype: object

In [8]:
sample_play.shape

(4955455, 7)

In [5]:

start = '2017-03-01'
end = '2017-04-29'
st,ed = pd.to_datetime(start), pd.to_datetime(end)
elapsed = ed-st

if elapsed > dt.timedelta(days = 1):
    print("Slept for > 1 day")

Slept for > 1 day


In [9]:
log_types = ['play', 'search', 'download']
window_list = [1,3,7,14,21]
snapshot_date = '2017-04-29'

In [10]:
def get_freq_avg(df, log_type, window, snapshot_date):
    snapshot_date = pd.to_datetime(snapshot_date)
    df2 = df.loc[(snapshot_date - df['date'])<= dt.timedelta(window), :]
    df_feature = df2.groupby('uid')['play_time'] \
                    .agg(['count', 'mean']) \
                    .rename(columns = {'count':'freq_' + log_type + '_last_' + str(window) + '_days',
                                       'mean' : 'avg_'+ log_type + 'time' + '_last_' + str(window) + '_days'})
    #df_feature = df_feature.rename(columns = {'date':})
    return df_feature

In [11]:
test1 = get_freq_avg(sample_play, 'play', window_list[0], snapshot_date)

In [12]:
print(test1.columns, test1.shape[0])
test1.head()

Index(['freq_play_last_1_days', 'avg_playtime_last_1_days'], dtype='object') 2791


,freq_play_last_1_days,avg_playtime_last_1_days
uid,,
1901839.0,3,222.333333
2285308.0,11,107.818182
5597959.0,6,111.166667
7533076.0,4,198.000000
7910960.0,14,214.142857


In [13]:
def get_all_freq_avg(df, log_type, window_list, snapshot_time):
    type_feature = get_freq_avg(df, log_type, window_list[0], snapshot_date)
    for window in window_list[1:]:
        #what = 'freq and average time of' + log_type + '_last_' + str(window) + '_days'
        #print("Calculating", what)
        test_what = get_freq_avg(df, log_type, window, snapshot_date)
        #print(test_what.head())
        type_feature = pd.merge(test_what, type_feature, how='left', left_index=True, right_index=True)
    return type_feature

In [14]:
play_feature = get_all_freq_avg(sample_play, 'play', window_list, snapshot_date)
#memory error ┓( ´∀` )┏

In [15]:
print(play_feature.shape)
play_feature.sample(8)

(5288, 10)


,freq_play_last_21_days,avg_playtime_last_21_days,freq_play_last_14_days,avg_playtime_last_14_days,freq_play_last_7_days,avg_playtime_last_7_days,freq_play_last_3_days,avg_playtime_last_3_days,freq_play_last_1_days,avg_playtime_last_1_days
uid,,,,,,,,,,
167754432.0,828,168.347826,552.0,156.530797,274.0,147.536496,96.0,141.031250,15.0,204.600000
168097696.0,487,230.030801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168437440.0,1241,1947.092667,1083.0,1770.493998,85.0,146.282353,18.0,138.055556,12.0,91.666667
168266560.0,394,215.532995,120.0,165.016667,30.0,184.466667,NaN,NaN,NaN,NaN
168019680.0,602,156.431894,403.0,171.292804,253.0,157.094862,120.0,181.508333,26.0,220.038462
57545228.0,30,171.866667,5.0,207.800000,1.0,61.000000,1.0,61.000000,1.0,61.000000
167624816.0,16,8.187500,16.0,8.187500,NaN,NaN,NaN,NaN,NaN,NaN
167935584.0,206,198.014563,123.0,222.747967,19.0,511.315789,NaN,NaN,NaN,NaN


In [16]:
print(len(sample_play.uid.unique()), play_feature.shape[0])

8823 5288


# why the number of uid drop after generating freq feature??

### 1.2. device feature

In [18]:
sample_play[['uid', 'device', 'Churn']].drop_duplicates().shape

(12618, 3)

In [19]:
sample_play[['uid', 'Churn']].drop_duplicates().shape

(8823, 2)

Adding device as another groupby condition causing significant increase, we can assume that some users have MusicBox installed in more than one device, such as their android phone and ipad.

Create a new value for user who have both device

In [21]:
device_count = sample_play[['uid', 'device']].drop_duplicates() \
                                              .groupby('uid')['device'] \
                                              .agg(['max', 'count'])

In [100]:
device_count.head()

,max,count
uid,,
400904.0,ar,1
838955.0,ar,1
971787.0,ar,1
1264383.0,ar,1
1901839.0,ar,1


In [22]:
device_count['device'] = np.where(device_count['count'] == 1, device_count['max'], 'both')

In [23]:
device_count['device'].value_counts()

ar      4862
both    3795
ip       166
Name: device, dtype: int64

As we assumed, there are 40% user have both device

In [24]:
device_feature = sample_play[['uid', 'Churn']].drop_duplicates() \
                                               .reset_index(drop=True) \
                                               .set_index('uid') \
                                               .join(device_count['device'])

In [26]:
print(device_feature.shape)
device_feature.sample(8)

(8823, 2)


,Churn,device
uid,,
168540016.0,0.0,ar
154617296.0,1.0,both
154613056.0,1.0,both
168764656.0,0.0,both
154795248.0,1.0,both
96635472.0,0.0,ar
167999856.0,0.0,both
154549488.0,1.0,both


### 1.3. merge device_feature and play_feature

In [75]:
play_feature_all = play_feature.join(device_feature,how = "left")
print(play_feature_all.shape)
play_feature_all.sample(8)


(5288, 12)


,freq_play_last_21_days,avg_playtime_last_21_days,freq_play_last_14_days,avg_playtime_last_14_days,freq_play_last_7_days,avg_playtime_last_7_days,freq_play_last_3_days,avg_playtime_last_3_days,freq_play_last_1_days,avg_playtime_last_1_days,Churn,device
uid,,,,,,,,,,,,
168514064.0,2649,128.042658,1626.0,128.148216,787.0,132.518424,414.0,127.615942,107.0,168.411215,0.0,ar
167796016.0,50,96.600000,15.0,76.466667,NaN,NaN,NaN,NaN,NaN,NaN,1.0,ar
166472064.0,149,62.604027,16.0,119.687500,5.0,81.200000,5.0,81.200000,5.0,81.200000,1.0,ar
167643024.0,322,172.763975,105.0,146.904762,58.0,190.517241,2.0,15.000000,NaN,NaN,1.0,ar
168123712.0,627,177.786284,355.0,212.997183,308.0,226.045455,299.0,229.969900,4.0,141.750000,0.0,both
168448688.0,88,122.352273,7.0,346.285714,NaN,NaN,NaN,NaN,NaN,NaN,1.0,both
168945072.0,305,79.206557,264.0,76.219697,46.0,83.739130,32.0,85.375000,32.0,85.375000,0.0,ar
168236384.0,157,235.783439,62.0,228.854839,41.0,224.292683,27.0,229.740741,22.0,265.181818,0.0,ar


### 1.4 complete_degree
check average music play complete degree of each user: avg(play_time/song_length)


In [29]:
sample_play.sample(8)

,uid,device,song_id,play_time,song_length,date,Churn
4428001,168738928.0,ip,658147.0,0.0,298.0,2017-04-10,1.0
4458463,154485536.0,ip,20794644.0,204.0,204.0,2017-03-04,1.0
385254,168584224.0,ar,17081440.0,0.0,333.0,2017-03-31,0.0
4089684,167957136.0,ar,6748381.0,1209.0,1209.0,2017-04-11,0.0
3777979,167845184.0,ar,1381671.0,12.0,166.0,2017-04-24,0.0
1931891,168217120.0,ip,4858877.0,86.0,195.0,2017-04-25,0.0
1890034,165995456.0,ar,22871364.0,21.0,86.0,2017-04-01,1.0
469288,167887840.0,ar,7103520.0,4.0,280.0,2017-04-01,0.0


In [60]:
music_complete = sample_play.groupby('uid')[['play_time', 'song_length']].mean()
print('How many user play time =0:', music_complete[music_complete.play_time ==0].count())
zero_uid = music_complete[music_complete.play_time ==0].index.tolist()
#if these users' average play time is 0....drop them? when clean play.log, I set play_time >=0, because I think for one song, the user
#just skip it, but not for all songs. If so, those user should be dropped.
print(zero_uid)
print(music_complete.head())

How many user play time =0: play_time      25
song_length    25
dtype: int64
[400904.0, 4334792.0, 8955891.0, 18205710.0, 61622544.0, 67449880.0, 106513760.0, 116009376.0, 124488288.0, 127544832.0, 128475592.0, 133089264.0, 135697680.0, 153251792.0, 154314656.0, 154882240.0, 155123760.0, 155320656.0, 155400256.0, 158236640.0, 161995536.0, 166548304.0, 167466016.0, 167848496.0, 169048464.0]
            play_time  song_length
uid                               
400904.0     0.000000   238.240000
838955.0   167.769231   222.307692
971787.0    32.500000   243.000000
1264383.0   87.666667   234.666667
1901839.0  160.455806   198.656846


In [121]:
complete_precent = music_complete.play_time/music_complete.song_length
complete_precent = pd.DataFrame(complete_precent, columns = ['precent'])
complete_precent.reset_index(inplace = True)
complete_precent.head()
#there are 25 user with play_time =0, 928 user with play/song_length >1.

,uid,precent
0,400904.0,0.000000
1,838955.0,0.754671
2,971787.0,0.133745
3,1264383.0,0.373580
4,1901839.0,0.807703


### 1.4. save play_feature_all

In [76]:
play_feature_all.to_csv("D:/MusicFile/play_feature_316.csv")

## 2. for search and download.log
### 2.1 frequency feature generation for search

In [78]:
sampled_search = pd.read_csv("D:/MusicFile/sample_search_316.csv")
sampled_search.sample(8)

,uid,device,file_name,date,record_time
251201,168930432.0,ar,20170423_3_search.log,2017-04-23T00:00:00.000-04:00,2017-04-23T18:13:34.000-04:00
9317,167656192.0,ip,20170331_1_search.log,2017-03-31T00:00:00.000-04:00,2017-03-31T19:50:34.000-04:00
289063,168733904.0,ar,20170330_3_search.log,2017-03-30T00:00:00.000-04:00,2017-03-30T23:17:28.000-04:00
160479,168958720.0,ip,20170413_3_search.log,2017-04-13T00:00:00.000-04:00,2017-04-14T00:10:47.000-04:00
260034,168328640.0,ar,20170422_2_search.log,2017-04-22T00:00:00.000-04:00,2017-04-22T07:48:51.000-04:00
459521,168980560.0,ar,20170331_3_search.log,2017-03-31T00:00:00.000-04:00,2017-03-31T20:59:27.000-04:00
404774,167814544.0,ar,20170415_1_search.log,2017-04-15T00:00:00.000-04:00,2017-04-15T22:22:59.000-04:00
244018,168777920.0,ar,20170408_3_search.log,2017-04-08T00:00:00.000-04:00,2017-04-08T21:53:17.000-04:00


In [79]:
sampled_search['date'] = pd.to_datetime(sampled_search['date'].str.slice(0,10))

In [80]:
sampled_search.sample(8)

,uid,device,file_name,date,record_time
174544,168032048.0,ar,20170405_1_search.log,2017-04-05,2017-04-05T10:43:39.000-04:00
331657,154564032.0,ar,1_1_search.log,2017-03-01,2017-03-01T16:10:14.000-05:00
93294,154794272.0,ar,1_1_search.log,2017-03-01,2017-03-01T23:29:15.000-05:00
468830,168879056.0,ar,20170330_3_search.log,2017-03-30,2017-03-30T20:14:16.000-04:00
337112,169040624.0,ar,20170419_3_search.log,2017-04-19,2017-04-19T09:31:00.000-04:00
367342,154662560.0,ar,1_1_search.log,2017-03-01,2017-03-01T19:13:47.000-05:00
99225,168376224.0,ar,20170420_2_search.log,2017-04-20,2017-04-20T18:24:33.000-04:00
213695,167907184.0,ar,20170416_1_search.log,2017-04-16,2017-04-16T23:44:42.000-04:00


In [6]:
sampled_search.dtypes

Unnamed: 0              int64
uid                   float64
device                 object
date           datetime64[ns]
record_time            object
dtype: object

In [84]:
sampled_search = sampled_search[['uid', 'device', 'date']]
len(sampled_search.uid.unique())

7714

In [85]:
sampled_search.to_csv("D:/MusicFile/sampled_search_316new.csv", header = True)

In [ ]:
'''
def get_freq_avg(df, log_type, window, snapshot_date):
    snapshot_date = pd.to_datetime(snapshot_date)
    df2 = df.loc[(snapshot_date - df['date'])<= dt.timedelta(window), :]
    df_feature = df2.groupby('uid')['play_time'] \
    .agg(['count', 'mean']) \
    .rename(columns = {'count':'freq_' + log_type + '_last_' + str(window) + '_days',
                       'mean' : 'avg_'+ log_type + 'time' + '_last_' + str(window) + '_days'})
    return df_feature

def get_all_freq(df, log_type, window_list, snapshot_time):
    type_feature = get_freq_avg(df, log_type, window_list[0], snapshot_date)
    for window in window_list[1:]:
        test_what = get_freq_avg(df, log_type, window, snapshot_date
        type_feature = pd.merge(test_what, type_feature, how='left', left_index=True, right_index=True)
    return type_feature


log_types = ['play', 'search', 'download']
window_list = [1,3,7,14,21]
snapshot_date = '2017-04-29'

'''


In [86]:
log_types = ['play', 'search', 'download']
window_list = [1,3,7,14,21]
snapshot_date = '2017-04-29'

In [87]:
def get_freq(df, log_type, window, snapshot_time):
    snapshot_date = pd.to_datetime(snapshot_time)
    df2 = df.loc[(snapshot_date - df['date'])<= dt.timedelta(window), :]
    df_feature = df2.groupby('uid')['date'] \
                    .agg(['count']) \
                    .rename(columns = {'count':'freq_' + log_type + '_last_' + str(window) + '_days'})
    
    return df_feature

In [9]:
sest = get_freq(sampled_search, 'search', window_list[0], snapshot_date)
sest.head()

,freq_search_last_1_days
uid,
1901839.0,5
2163514.0,1
2285308.0,1
3752046.0,3
5597959.0,18


In [88]:
def get_all_freq(df, log_type, window_list, snapshot_time):
    type_feature = get_freq(df, log_type, window_list[0], snapshot_date)
    for window in window_list[1:]:
        test_what = get_freq(df, log_type, window, snapshot_date)
        type_feature = pd.merge(test_what, type_feature, how='left', left_index=True, right_index=True)
    return type_feature

In [89]:
search_feature = get_all_freq(sampled_search, 'search', window_list, snapshot_date)

In [91]:
search_feature.sample(8)
#notice: uid now is the row index, we need it as a column uid, but when save to csv, it automatically change to a column

,freq_search_last_21_days,freq_search_last_14_days,freq_search_last_7_days,freq_search_last_3_days,freq_search_last_1_days
uid,,,,,
154383536.0,1,1.0,1.0,NaN,NaN
168291328.0,183,155.0,118.0,94.0,94.0
169127536.0,20,16.0,6.0,6.0,6.0
164304880.0,18,16.0,16.0,16.0,16.0
168250144.0,65,53.0,40.0,36.0,35.0
168078656.0,12,7.0,6.0,6.0,6.0
168646448.0,32,24.0,11.0,11.0,11.0
168926224.0,110,110.0,86.0,76.0,73.0


In [92]:
search_feature.shape

(4633, 5)

In [93]:
search_feature.to_csv("D:/MusicFile/search_feature_316.csv", header = True)

### 2.2 frequency feature generation for download

In [95]:
sampled_download = pd.read_csv("D:/MusicFile/sample_download_316.csv")
sampled_download['date'] = pd.to_datetime(sampled_download['date'].str.slice(0,10))
sampled_download.sample(8)

,uid,device,song_id,date
250139,168495920.0,ar,94675.0,2017-04-26
206818,168714048.0,ar,6368802.0,2017-04-16
73410,168471200.0,ar,3197116.0,2017-04-03
218557,168948688.0,ar,4269836.0,2017-05-11
204064,168794400.0,ar,519479.0,2017-03-31
199924,167952240.0,ar,5070737.0,2017-05-09
201782,49423096.0,ar,4799485.0,2017-04-07
279274,168518592.0,ar,3328045.0,2017-03-30


In [16]:
dest = get_freq(sampled_download, 'download', window_list[0], snapshot_date)
dest.head()

,freq_download_last_1_days
uid,
2280694.0,5
3752046.0,1
7910960.0,3
8853855.0,2
14780539.0,27


In [96]:
download_feature = get_all_freq(sampled_download, 'download', window_list, snapshot_date)

In [98]:
download_feature.sample(8)

,freq_download_last_21_days,freq_download_last_14_days,freq_download_last_7_days,freq_download_last_3_days,freq_download_last_1_days
uid,,,,,
167662048.0,9,8.0,7.0,7.0,2.0
168883456.0,80,59.0,36.0,22.0,4.0
167726464.0,8,7.0,6.0,6.0,2.0
167643488.0,164,164.0,164.0,164.0,164.0
167958544.0,1,1.0,1.0,NaN,NaN
168196624.0,38,38.0,8.0,8.0,8.0
168695184.0,8,8.0,6.0,6.0,6.0
168239504.0,6,5.0,3.0,3.0,3.0


In [99]:
download_feature.shape

(3899, 5)

In [100]:
download_feature.to_csv("D:/MusicFile/download_feature_316.csv", header = True)

In [38]:
#join search and download
ds = pd.merge(search_feature, download_feature, left_on = 'uid', right_on = 'uid',  how = 'left')
ds.sample(10)

,uid,freq_search_last_21_days,freq_search_last_14_days,freq_search_last_7_days,freq_search_last_3_days,freq_search_last_1_days,freq_download_last_21_days,freq_download_last_14_days,freq_download_last_7_days,freq_download_last_3_days,freq_download_last_1_days
3628,168710656.0,3,NaN,NaN,NaN,NaN,6.0,2.0,NaN,NaN,NaN
3654,168719200.0,17,14.0,10.0,8.0,8.0,11.0,8.0,4.0,3.0,3.0
130,125439600.0,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4513,169007776.0,58,48.0,40.0,39.0,35.0,162.0,153.0,152.0,10.0,9.0
3093,168515184.0,35,10.0,8.0,6.0,6.0,4.0,NaN,NaN,NaN,NaN
2288,168248544.0,8,5.0,5.0,5.0,5.0,31.0,2.0,2.0,2.0,2.0
3017,168489712.0,18,13.0,11.0,8.0,8.0,4.0,3.0,3.0,3.0,3.0
2735,168397136.0,15,1.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN
3874,168789456.0,14,12.0,7.0,1.0,1.0,7.0,5.0,5.0,1.0,NaN
1960,168130448.0,90,65.0,37.0,28.0,28.0,43.0,9.0,8.0,8.0,8.0


### 3.Join play, search, download feature
Since our analysis based on play, so if user doesn't have search, download, just plugged in NaN
play_feature_all has 5395 user, search has 4645, download has 3906

In [22]:
print(4645/5395, 3906/5395)

0.860982391102873 0.7240037071362373


In [101]:
#reload type_feature

play_feature_all = pd.read_csv("D:/MusicFile/play_feature_316.csv")
search_feature = pd.read_csv("D:/MusicFile/search_feature_316.csv")
download_feature = pd.read_csv("D:/MusicFile/download_feature_316.csv")

In [102]:
print("play_feature:",play_feature_all.shape)
print("search_feature:",search_feature.shape)
print("download_feature:",download_feature.shape)

play_feature: (5288, 13)
search_feature: (4633, 6)
download_feature: (3899, 6)


In [107]:
all_feature = play_feature_all.merge(search_feature, on = 'uid', how = "left") \
                              .merge(download_feature, on = 'uid', how = 'left')
    

In [108]:
all_feature.sample(10)    

,uid,freq_play_last_21_days,avg_playtime_last_21_days,freq_play_last_14_days,avg_playtime_last_14_days,freq_play_last_7_days,avg_playtime_last_7_days,freq_play_last_3_days,avg_playtime_last_3_days,freq_play_last_1_days,...,freq_search_last_21_days,freq_search_last_14_days,freq_search_last_7_days,freq_search_last_3_days,freq_search_last_1_days,freq_download_last_21_days,freq_download_last_14_days,freq_download_last_7_days,freq_download_last_3_days,freq_download_last_1_days
537,167544512.0,482,192.412863,407.0,188.707617,225.0,161.644444,140.0,199.400000,70.0,...,104.0,101.0,96.0,90.0,88.0,NaN,NaN,NaN,NaN,NaN
3233,168406624.0,673,29354.882615,217.0,41313.304147,99.0,28996.121212,65.0,44068.338462,17.0,...,28.0,19.0,17.0,8.0,5.0,31.0,19.0,16.0,1.0,NaN
3794,168574960.0,390,152.451282,336.0,153.794643,127.0,155.244094,66.0,135.590909,30.0,...,55.0,49.0,16.0,2.0,1.0,15.0,15.0,4.0,2.0,2.0
2006,168009488.0,1594,83.992472,696.0,154.058908,374.0,139.708556,42.0,104.000000,4.0,...,44.0,39.0,22.0,18.0,18.0,26.0,23.0,12.0,4.0,4.0
4290,168742512.0,116,177.112069,92.0,209.554348,46.0,56.347826,NaN,NaN,NaN,...,24.0,21.0,20.0,15.0,15.0,3.0,1.0,NaN,NaN,NaN
1038,167711952.0,10,161.700000,4.0,66.500000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3259,168414080.0,979,159.421859,738.0,161.636856,478.0,176.343096,84.0,154.892857,7.0,...,79.0,76.0,67.0,40.0,33.0,102.0,36.0,25.0,13.0,12.0
2004,168009216.0,229,197.445415,172.0,208.011628,48.0,204.541667,42.0,219.428571,28.0,...,12.0,5.0,5.0,5.0,3.0,NaN,NaN,NaN,NaN,NaN
4605,168836208.0,149,113.281879,141.0,115.943262,66.0,193.500000,48.0,144.979167,33.0,...,23.0,23.0,2.0,2.0,2.0,37.0,34.0,2.0,2.0,2.0
1212,167760992.0,2689,71.711045,2603.0,69.956204,1549.0,68.593932,810.0,60.007407,72.0,...,69.0,67.0,41.0,36.0,33.0,11.0,11.0,10.0,8.0,4.0


In [105]:
all_feature.shape

(5288, 23)

In [109]:
all_feature.isnull().sum()

uid                              0
freq_play_last_21_days           0
avg_playtime_last_21_days        0
freq_play_last_14_days         302
avg_playtime_last_14_days      302
freq_play_last_7_days          814
avg_playtime_last_7_days       814
freq_play_last_3_days         1481
avg_playtime_last_3_days      1481
freq_play_last_1_days         2497
avg_playtime_last_1_days      2497
Churn                            0
device                           0
freq_search_last_21_days       701
freq_search_last_14_days       946
freq_search_last_7_days       1283
freq_search_last_3_days       1513
freq_search_last_1_days       1608
freq_download_last_21_days    1418
freq_download_last_14_days    1793
freq_download_last_7_days     2223
freq_download_last_3_days     2497
freq_download_last_1_days     2652
dtype: int64

In [110]:
all_feature.columns

Index(['uid', 'freq_play_last_21_days', 'avg_playtime_last_21_days',
       'freq_play_last_14_days', 'avg_playtime_last_14_days',
       'freq_play_last_7_days', 'avg_playtime_last_7_days',
       'freq_play_last_3_days', 'avg_playtime_last_3_days',
       'freq_play_last_1_days', 'avg_playtime_last_1_days', 'Churn', 'device',
       'freq_search_last_21_days', 'freq_search_last_14_days',
       'freq_search_last_7_days', 'freq_search_last_3_days',
       'freq_search_last_1_days', 'freq_download_last_21_days',
       'freq_download_last_14_days', 'freq_download_last_7_days',
       'freq_download_last_3_days', 'freq_download_last_1_days'],
      dtype='object')

In [111]:
#reorder columns
all_feature = all_feature[['uid','device', 'freq_play_last_21_days', 'avg_playtime_last_21_days',
       'freq_play_last_14_days', 'avg_playtime_last_14_days',
       'freq_play_last_7_days', 'avg_playtime_last_7_days',
       'freq_play_last_3_days', 'avg_playtime_last_3_days',
       'freq_play_last_1_days', 'avg_playtime_last_1_days', 
       'freq_search_last_21_days', 'freq_search_last_14_days',
       'freq_search_last_7_days', 'freq_search_last_3_days',
       'freq_search_last_1_days', 'freq_download_last_21_days',
       'freq_download_last_14_days', 'freq_download_last_7_days',
       'freq_download_last_3_days', 'freq_download_last_1_days', 'Churn']]

Since all NAN appears in search/download feature, NaN means there is no such record during this period, fillin with 0 is doable.

In [112]:
all_feature = all_feature.fillna(0)

## One_hot_encoding for device
sklearn can not accept numeric variables, for device, either use one-hot encoding or try model without device, save both and try later.

In [113]:
device_dummies = pd.get_dummies(all_feature.device)

In [85]:
device_dummies.sample(10)

,ar,both,ip
3873,0,1,0
2719,0,1,0
1573,0,1,0
2758,1,0,0
784,1,0,0
1372,1,0,0
4656,1,0,0
4631,1,0,0
2820,1,0,0
4597,1,0,0


In [114]:
#merge to the all_feature as all_feature_device_dummy
all_feature['device_ar'], all_feature['device_both'], all_feature['device_ip'] = device_dummies.ar, device_dummies.both, device_dummies.ip
all_feature_dd = all_feature.drop('device', 1)


In [115]:
all_feature_dd.sample(5)

,uid,freq_play_last_21_days,avg_playtime_last_21_days,freq_play_last_14_days,avg_playtime_last_14_days,freq_play_last_7_days,avg_playtime_last_7_days,freq_play_last_3_days,avg_playtime_last_3_days,freq_play_last_1_days,...,freq_search_last_1_days,freq_download_last_21_days,freq_download_last_14_days,freq_download_last_7_days,freq_download_last_3_days,freq_download_last_1_days,Churn,device_ar,device_both,device_ip
2767,168258848.0,85,171.341176,68.0,177.647059,20.0,182.350000,0.0,0.000000,0.0,...,0.0,24.0,5.0,4.0,4.0,4.0,0.0,0,1,0
4290,168742512.0,116,177.112069,92.0,209.554348,46.0,56.347826,0.0,0.000000,0.0,...,15.0,3.0,1.0,0.0,0.0,0.0,0.0,0,1,0
393,164388416.0,194,138.551546,50.0,211.780000,9.0,191.111111,9.0,191.111111,9.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0,0
630,167590480.0,785,129.098089,216.0,128.004630,9.0,181.888889,0.0,0.000000,0.0,...,0.0,4.0,2.0,0.0,0.0,0.0,1.0,1,0,0
4225,168719200.0,692,226.731214,438.0,240.568493,241.0,239.489627,136.0,234.227941,89.0,...,8.0,11.0,8.0,4.0,3.0,3.0,0.0,1,0,0


In [116]:
#drop device as all_feature_no_d
all_feature_nd = all_feature_dd.drop(['device_ar', 'device_ip', 'device_both'],1)
all_feature_nd.sample(5)

,uid,freq_play_last_21_days,avg_playtime_last_21_days,freq_play_last_14_days,avg_playtime_last_14_days,freq_play_last_7_days,avg_playtime_last_7_days,freq_play_last_3_days,avg_playtime_last_3_days,freq_play_last_1_days,...,freq_search_last_14_days,freq_search_last_7_days,freq_search_last_3_days,freq_search_last_1_days,freq_download_last_21_days,freq_download_last_14_days,freq_download_last_7_days,freq_download_last_3_days,freq_download_last_1_days,Churn
714,167615216.0,179,191.234637,14.0,87.285714,12.0,75.750000,12.0,75.750000,0.0,...,6.0,6.0,6.0,6.0,221.0,2.0,2.0,2.0,2.0,0.0
4040,168656368.0,7,110.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3892,168610240.0,209,260.363636,167.0,219.874251,119.0,240.974790,71.0,216.929577,25.0,...,8.0,6.0,6.0,5.0,2.0,2.0,1.0,1.0,0.0,0.0
1294,167788992.0,910,196.486813,673.0,197.575037,355.0,203.991549,106.0,245.707547,53.0,...,90.0,53.0,44.0,40.0,26.0,20.0,11.0,11.0,11.0,0.0
4986,168957040.0,757,184.690885,146.0,95.205479,29.0,86.241379,2.0,191.500000,1.0,...,4.0,4.0,4.0,4.0,34.0,32.0,32.0,32.0,32.0,0.0


# different version of features

In [123]:
#add precentage
all_feature_total = all_feature_dd.merge(complete_precent, on = 'uid', how = 'inner')
print(len(all_feature_total.uid.unique()))
all_feature_total.sample(8)

5288


,uid,freq_play_last_21_days,avg_playtime_last_21_days,freq_play_last_14_days,avg_playtime_last_14_days,freq_play_last_7_days,avg_playtime_last_7_days,freq_play_last_3_days,avg_playtime_last_3_days,freq_play_last_1_days,...,freq_download_last_21_days,freq_download_last_14_days,freq_download_last_7_days,freq_download_last_3_days,freq_download_last_1_days,Churn,device_ar,device_both,device_ip,precent
3631,168528304.0,463,119.833693,328.0,121.295732,212.0,118.231132,140.0,105.578571,2.0,...,31.0,29.0,29.0,0.0,0.0,0.0,0,1,0,0.490147
3932,168622256.0,563,201.380107,295.0,232.437288,133.0,264.691729,63.0,284.603175,32.0,...,28.0,24.0,24.0,24.0,24.0,0.0,1,0,0,0.747346
2133,168052560.0,3,231.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,8.0,8.0,8.0,8.0,8.0,0.0,0,1,0,0.679533
4291,168742720.0,177,292.864407,127.0,370.992126,62.0,467.064516,41.0,374.902439,20.0,...,4.0,4.0,4.0,4.0,4.0,0.0,1,0,0,0.279409
3486,168484480.0,540,110.764815,302.0,107.701987,160.0,132.131250,64.0,156.750000,7.0,...,34.0,20.0,20.0,19.0,19.0,0.0,0,1,0,0.441160
4326,168753328.0,152,241.421053,80.0,250.400000,69.0,253.028986,27.0,260.259259,0.0,...,7.0,7.0,6.0,5.0,3.0,0.0,0,1,0,1.027988
2668,168228320.0,456,162.809211,341.0,152.991202,95.0,149.600000,53.0,208.886792,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0.778997
491,167073008.0,74,57.945946,74.0,57.945946,74.0,57.945946,74.0,57.945946,74.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1,0,0,0.228389


In [124]:
all_feature_dd.to_csv("D:/MusicFile/all_feature_dd_316.csv", header = True, index = False)

In [125]:
all_feature_nd.to_csv("D:/MusicFile/all_feature_nd_316.csv", header = True, index = False)

In [126]:
all_feature_total.to_csv("D:/MusicFile/all_feature_total_316.csv", header = True, index = False)